## testing hw

In [2]:
import numpy as np
import pandas as pd
from scipy import sparse
from cf import *

## Encoding rating data
Here are our very small subset of fake data to get us started.

In [3]:
# The first row says that user 1 reated movie 11 with a score of 4
!cat tiny_training.csv 

userId,movieId,rating
11,1,4
11,23,5
2,23,5
2,4,3
31,1,4
31,23,4
4,1,5
4,3,2
52,1,1
52,3,4
61,3,5
7,23,1
7,3,3


In [4]:
df = pd.read_csv("tiny_training.csv")
df, num_users, num_movies = encode_data(df)

In [5]:
df

,userId,movieId,rating
0,0,0,4
1,0,1,5
2,1,1,5
3,1,2,3
4,2,0,4
5,2,1,4
6,3,0,5
7,3,3,2
8,4,0,1
9,4,3,4


In [6]:
assert(num_users == 7)

In [7]:
assert(num_movies == 4)

In [8]:
np.testing.assert_equal(df["userId"].values, np.array([0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 6, 6]))

In [9]:
np.testing.assert_equal(df["movieId"].values, np.array([0, 1, 1, 2, 0, 1, 0, 3, 0, 3, 3, 1, 3]))

## Initializing parameters

In [10]:
# here is an example on how the prediction matrix would look like with 7 users and 5 movies
np.dot(create_embedings(7,3), create_embedings(5,3).transpose())

array([[3.55790894, 4.69774849, 0.92361109, 1.58739544, 3.00593239],
       [4.69774849, 7.44656163, 1.18135616, 2.64524868, 4.74559066],
       [0.92361109, 1.18135616, 0.24548062, 0.34025121, 0.69616965],
       [1.58739544, 2.64524868, 0.34025121, 1.61561   , 2.41361975],
       [3.00593239, 4.74559066, 0.69616965, 2.41361975, 3.82505541],
       [2.02000808, 3.29656257, 0.43174569, 2.065911  , 3.07264619],
       [2.07691001, 3.02887291, 0.53270924, 1.02482544, 1.90251125]])

## Encoding Y as a sparse matrix
This code helps you encode a $Y$ as a sparse matrix from the dataframe. 

In [11]:
df = pd.read_csv("tiny_training.csv")
df, num_users, num_movies = encode_data(df)
Y = df2matrix(df, num_users, num_movies)

In [12]:
print(Y)

  (0, 0)	4
  (2, 0)	4
  (3, 0)	5
  (4, 0)	1
  (0, 1)	5
  (1, 1)	5
  (2, 1)	4
  (6, 1)	1
  (1, 2)	3
  (3, 3)	2
  (4, 3)	4
  (5, 3)	5
  (6, 3)	3


## Calculating the cost function

In [13]:
emb_user = np.ones((num_users, 3))
emb_movie = np.ones((num_movies, 3))
error = cost(df, emb_user, emb_movie)
assert(np.around(error, decimals=2) == 2.23)

## Calculating gradient

In [14]:
K = 3
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0])
grad_user, grad_movie = gradient(df, Y, emb_user, emb_movie)

In [15]:
user=1
approx = np.array([finite_difference(df, emb_user, emb_movie, ind_u=user, k=i) for i in range(K)])
assert(np.all(np.abs(grad_user[user] - approx) < 0.0001))

In [16]:
movie=1
approx = np.array([finite_difference(df, emb_user, emb_movie, ind_m=movie, k=i) for i in range(K)])
assert(np.all(np.abs(grad_movie[movie] - approx) < 0.0001))

## Gradient descent with momentum

In [17]:
emb_user = create_embedings(num_users, 3)
emb_movie = create_embedings(num_movies, 3)
emb_user, emb_movie = gradient_descent(df, emb_user, emb_movie, iterations=200, learning_rate=0.01)

At 0th iteration, the training cost is 4.75400712709481
At 50th iteration, the training cost is 1.957651064221775
At 100th iteration, the training cost is 1.0458015647845733
At 150th iteration, the training cost is 0.7333410655387985


In [18]:
train_mse = cost(df, emb_user, emb_movie)
assert(np.around(train_mse, decimals=2) == 0.55)

## Predicting on new data
Now we should write a function that given new data is able to predict ratings. First we write a function that encodes new data. If a new user or item is present that row should be remove. Collaborative Filtering is not good at handling new users or new items. To help with this task, you could write a an auxiliary function similar to `proc_col`.

In [19]:
df_t = pd.read_csv("tiny_training.csv")
df_v = pd.read_csv("tiny_val.csv")
df_v = encode_new_data(df_v, df_t)

In [20]:
assert(len(df_v.userId.unique())==2)

In [21]:
assert(len(df_v) == 2)

## Putting it all together
For this part you should get data from here
`wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip`

In [26]:
# sorting by timestamp
path = "/Users/ouyangjiaxuan/Downloads/Prework/ad_ml_hw1/ml-latest-small/"
data = pd.read_csv(path + "ratings.csv")
data = data.sort_values(by=['timestamp'])

In [27]:
data.head()

,userId,movieId,rating,timestamp
66719,429,595,5.0,828124615
66716,429,588,5.0,828124615
66717,429,590,5.0,828124615
66718,429,592,5.0,828124615
66712,429,432,3.0,828124615


In [28]:
m = int(data.shape[0]*0.8)
data.shape, m

((100836, 4), 80668)

In [29]:
train = data[:m].copy()
val = data[m:].copy()
df_train, num_users, num_movies = encode_data(train.copy())
df_val = encode_new_data(val.copy(), train.copy())
print(len(val), len(df_val))

20168 1311


In [40]:
K = 50
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
emb_user, emb_movie = gradient_descent(df_train, emb_user, emb_movie, iterations=2000,
                                       learning_rate=1, df_val=df_val)

At 0th iteration, the training cost is 12.136404414670121
At 0th iteration, the validation cost is 12.310601296043618
At 50th iteration, the training cost is 9.613903397530699
At 50th iteration, the validation cost is 10.353878141282681
At 100th iteration, the training cost is 6.483520773751977
At 100th iteration, the validation cost is 8.351624227410639
At 150th iteration, the training cost is 4.519311775145557
At 150th iteration, the validation cost is 6.96925304302195
At 200th iteration, the training cost is 3.487453177675796
At 200th iteration, the validation cost is 6.135265489703383
At 250th iteration, the training cost is 2.8460807656602087
At 250th iteration, the validation cost is 5.541986970150605
At 300th iteration, the training cost is 2.411235646140604
At 300th iteration, the validation cost is 5.063727564992295
At 350th iteration, the training cost is 2.101033524207363
At 350th iteration, the validation cost is 4.659691742103504
At 400th iteration, the training cost is 1.

In [41]:
train_mse = cost(df_train, emb_user, emb_movie)
val_mse = cost(df_val, emb_user, emb_movie)
print(train_mse, val_mse)

0.7113513270477632 1.140599450059145


In [42]:
train_mse = cost(df_train, emb_user, emb_movie)
assert(np.around(train_mse, decimals=2) == 0.71)

In [43]:
val_mse = cost(df_val, emb_user, emb_movie)
assert(np.around(val_mse, decimals=2) == 1.14)

In [44]:
K = 50
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
emb_user, emb_movie = gradient_descent(df_train, emb_user, emb_movie, iterations=200, learning_rate=1, df_val=df_val)

At 0th iteration, the training cost is 12.136404414670121
At 0th iteration, the validation cost is 12.310601296043618
At 50th iteration, the training cost is 9.613903397530699
At 50th iteration, the validation cost is 10.353878141282681
At 100th iteration, the training cost is 6.483520773751977
At 100th iteration, the validation cost is 8.351624227410639
At 150th iteration, the training cost is 4.519311775145557
At 150th iteration, the validation cost is 6.96925304302195


In [45]:
train_mse = cost(df_train, emb_user, emb_movie)
val_mse = cost(df_val, emb_user, emb_movie)
print(train_mse, val_mse)

3.503321206458504 6.148866866928117


In [46]:
train_mse = cost(df_train, emb_user, emb_movie)
assert(np.around(train_mse, decimals=1) == 3.5)

In [47]:
val_mse = cost(df_val, emb_user, emb_movie)
assert(np.around(val_mse, decimals=1) == 6.1)